# Rekurrente nevrale nettverk

I det forrige modulen har vi brukt rike semantiske representasjoner av tekst, og en enkel lineær klassifikator på toppen av embeddingene. Det denne arkitekturen gjør, er å fange opp den aggregerte meningen av ordene i en setning, men den tar ikke hensyn til **rekkefølgen** av ordene, fordi aggregeringsoperasjonen på embeddingene fjerner denne informasjonen fra den opprinnelige teksten. Siden disse modellene ikke kan modellere ordrekkefølge, kan de ikke løse mer komplekse eller tvetydige oppgaver som tekstgenerering eller spørsmål-svar.

For å fange opp meningen i en tekstsekvens, må vi bruke en annen nevralt nettverksarkitektur, som kalles et **rekurrent nevralt nettverk**, eller RNN. I et RNN sender vi setningen vår gjennom nettverket ett symbol om gangen, og nettverket produserer en **tilstand**, som vi deretter sender tilbake til nettverket sammen med neste symbol.

Gitt inndatasekvensen av token $X_0,\dots,X_n$, lager RNN en sekvens av nevrale nettverksblokker og trener denne sekvensen ende-til-ende ved hjelp av backpropagation. Hver nettverksblokk tar et par $(X_i,S_i)$ som input og produserer $S_{i+1}$ som resultat. Den endelige tilstanden $S_n$ eller utgangen $X_n$ går inn i en lineær klassifikator for å produsere resultatet. Alle nettverksblokker deler de samme vektene og trenes ende-til-ende ved hjelp av én backpropagation-passering.

Siden tilstandsvektorene $S_0,\dots,S_n$ sendes gjennom nettverket, er det i stand til å lære de sekvensielle avhengighetene mellom ordene. For eksempel, når ordet *ikke* dukker opp et sted i sekvensen, kan det lære å negere visse elementer i tilstandsvektoren, noe som resulterer i negasjon.

> Siden vektene til alle RNN-blokkene på bildet er delte, kan det samme bildet representeres som én blokk (til høyre) med en rekurrent tilbakemeldingssløyfe, som sender utgangstilstanden til nettverket tilbake til inngangen.

La oss se hvordan rekurrente nevrale nettverk kan hjelpe oss med å klassifisere nyhetsdatasettet vårt.


In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## Enkel RNN-klassifiserer

Når det gjelder enkel RNN, er hver rekurrent enhet et enkelt lineært nettverk som tar en sammenkoblet inputvektor og tilstandsvektor, og produserer en ny tilstandsvektor. PyTorch representerer denne enheten med klassen `RNNCell`, og et nettverk av slike celler - som laget `RNN`.

For å definere en RNN-klassifiserer, vil vi først bruke et innebyggingslag for å redusere dimensjonaliteten til input-ordforrådet, og deretter ha et RNN-lag på toppen av dette:


In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **Note:** Vi bruker et utrent embedding-lag her for enkelhets skyld, men for enda bedre resultater kan vi bruke et forhåndstrent embedding-lag med Word2Vec eller GloVe embeddings, som beskrevet i den forrige enheten. For bedre forståelse kan det være lurt å tilpasse denne koden til å fungere med forhåndstrente embeddings.

I vårt tilfelle vil vi bruke en data loader med padding, slik at hver batch vil ha et antall sekvenser med samme lengde. RNN-laget vil ta sekvensen av embedding-tensore og produsere to utganger:
* $x$ er en sekvens av RNN-celleutganger ved hvert steg
* $h$ er en endelig skjult tilstand for det siste elementet i sekvensen

Deretter bruker vi en fullt tilkoblet lineær klassifiserer for å få antall klasser.

> **Note:** RNN-er er ganske vanskelige å trene, fordi når RNN-cellene rulles ut langs sekvenslengden, blir antallet lag involvert i tilbakepropagering ganske stort. Derfor må vi velge en liten læringsrate og trene nettverket på et større datasett for å oppnå gode resultater. Det kan ta ganske lang tid, så det er foretrukket å bruke GPU.


In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.3090625
6400: acc=0.38921875
9600: acc=0.4590625
12800: acc=0.511953125
16000: acc=0.5506875
19200: acc=0.57921875
22400: acc=0.6070089285714285
25600: acc=0.6304296875
28800: acc=0.6484027777777778
32000: acc=0.66509375
35200: acc=0.6790056818181818
38400: acc=0.6929166666666666
41600: acc=0.7035817307692308
44800: acc=0.7137276785714286
48000: acc=0.72225
51200: acc=0.73001953125
54400: acc=0.7372794117647059
57600: acc=0.7436631944444444
60800: acc=0.7503947368421052
64000: acc=0.75634375
67200: acc=0.7615773809523809
70400: acc=0.7662642045454545
73600: acc=0.7708423913043478
76800: acc=0.7751822916666666
80000: acc=0.7790625
83200: acc=0.7825
86400: acc=0.7858564814814815
89600: acc=0.7890513392857142
92800: acc=0.7920474137931034
96000: acc=0.7952708333333334
99200: acc=0.7982258064516129
102400: acc=0.80099609375
105600: acc=0.8037594696969697
108800: acc=0.8060569852941176


## Lang Korttidsminne (LSTM)

Et av hovedproblemene med klassiske RNN-er er det såkalte **problemet med forsvinnende gradienter**. Fordi RNN-er trenes ende-til-ende i én bakoverpropageringspassering, har de vanskeligheter med å propagere feil til de første lagene i nettverket, og dermed kan ikke nettverket lære relasjoner mellom fjerne tokens. En av måtene å unngå dette problemet på er å introdusere **eksplisitt tilstandshåndtering** ved å bruke såkalte **porter**. Det finnes to mest kjente arkitekturer av denne typen: **Lang Korttidsminne** (LSTM) og **Gated Relay Unit** (GRU).

![Bilde som viser et eksempel på en lang korttidsminnecelle](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

LSTM-nettverket er organisert på en måte som ligner på RNN, men det er to tilstander som blir sendt fra lag til lag: faktisk tilstand $c$, og skjult vektor $h$. Ved hver enhet blir skjult vektor $h_i$ sammenkoblet med input $x_i$, og de kontrollerer hva som skjer med tilstanden $c$ via **porter**. Hver port er et nevralt nettverk med sigmoid-aktivering (output i området $[0,1]$), som kan betraktes som en bitmaske når den multipliseres med tilstandsvektoren. Det finnes følgende porter (fra venstre til høyre på bildet ovenfor):
* **glemporten** tar skjult vektor og bestemmer hvilke komponenter av vektoren $c$ vi må glemme, og hvilke vi skal sende videre.
* **inputporten** tar noe informasjon fra input og skjult vektor, og setter det inn i tilstanden.
* **outputporten** transformerer tilstanden via et lineært lag med $\tanh$-aktivering, og velger deretter noen av komponentene ved hjelp av skjult vektor $h_i$ for å produsere ny tilstand $c_{i+1}$.

Komponentene i tilstanden $c$ kan betraktes som noen flagg som kan slås av og på. For eksempel, når vi møter et navn *Alice* i sekvensen, kan vi anta at det refererer til en kvinnelig karakter, og heve flagget i tilstanden som indikerer at vi har et kvinnelig substantiv i setningen. Når vi senere møter frasen *og Tom*, vil vi heve flagget som indikerer at vi har et flertallssubstantiv. Dermed kan vi ved å manipulere tilstanden antagelig holde styr på grammatiske egenskaper ved setningsdeler.

> **Note**: En flott ressurs for å forstå det indre av LSTM er denne utmerkede artikkelen [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) av Christopher Olah.

Selv om den interne strukturen til LSTM-cellen kan virke kompleks, skjuler PyTorch denne implementasjonen inne i `LSTMCell`-klassen, og tilbyr `LSTM`-objektet for å representere hele LSTM-laget. Dermed vil implementeringen av en LSTM-klassifiserer være ganske lik den enkle RNN-en vi har sett ovenfor:


In [4]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

In [5]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.259375
6400: acc=0.25859375
9600: acc=0.26177083333333334
12800: acc=0.2784375
16000: acc=0.313
19200: acc=0.3528645833333333
22400: acc=0.3965625
25600: acc=0.4385546875
28800: acc=0.4752777777777778
32000: acc=0.505375
35200: acc=0.5326704545454546
38400: acc=0.5557552083333334
41600: acc=0.5760817307692307
44800: acc=0.5954910714285714
48000: acc=0.6118333333333333
51200: acc=0.62681640625
54400: acc=0.6404779411764706
57600: acc=0.6520138888888889
60800: acc=0.662828947368421
64000: acc=0.673546875
67200: acc=0.6831547619047619
70400: acc=0.6917897727272727
73600: acc=0.6997146739130434
76800: acc=0.707109375
80000: acc=0.714075
83200: acc=0.7209134615384616
86400: acc=0.727037037037037
89600: acc=0.7326674107142858
92800: acc=0.7379633620689655
96000: acc=0.7433645833333333
99200: acc=0.7479032258064516
102400: acc=0.752119140625
105600: acc=0.7562405303030303
108800: acc=0.76015625
112000: acc=0.7641339285714286
115200: acc=0.7677777777777778
118400: acc=0.77112331081

(0.03487814127604167, 0.7728)

## Pakkede sekvenser

I vårt eksempel måtte vi fylle opp alle sekvensene i minibatchen med nullvektorer. Selv om dette fører til noe sløsing med minne, er det mer kritisk med RNN-er at ekstra RNN-celler opprettes for de utfylte input-elementene. Disse deltar i treningen, men inneholder ingen viktig inputinformasjon. Det ville vært mye bedre å trene RNN kun til den faktiske sekvensstørrelsen.

For å oppnå dette, er et spesielt format for lagring av utfylte sekvenser introdusert i PyTorch. Anta at vi har en utfylt minibatch som ser slik ut:
```
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
```
Her representerer 0 utfylte verdier, og den faktiske lengdevektoren for inputsekvensene er `[5,3,1]`.

For å effektivt trene RNN med utfylte sekvenser, ønsker vi å starte treningen med den første gruppen av RNN-celler med en stor minibatch (`[1,6,9]`), men deretter avslutte behandlingen av den tredje sekvensen og fortsette treningen med mindre minibatcher (`[2,7]`, `[3,8]`), og så videre. Dermed representeres den pakkede sekvensen som én vektor – i vårt tilfelle `[1,6,9,2,7,3,8,4,5]`, og en lengdevektor (`[5,3,1]`), som vi enkelt kan bruke til å rekonstruere den opprinnelige utfylte minibatchen.

For å produsere en pakket sekvens, kan vi bruke funksjonen `torch.nn.utils.rnn.pack_padded_sequence`. Alle rekurrente lag, inkludert RNN, LSTM og GRU, støtter pakkede sekvenser som input og produserer pakket output, som kan dekodes ved hjelp av `torch.nn.utils.rnn.pad_packed_sequence`.

For å kunne produsere en pakket sekvens, må vi sende lengdevektoren til nettverket, og derfor trenger vi en annen funksjon for å forberede minibatcher:


In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

Den faktiske nettverket vil være veldig likt `LSTMClassifier` ovenfor, men `forward`-passet vil motta både polstret minibatch og vektoren av sekvenslengder. Etter å ha beregnet embedding, beregner vi en pakket sekvens, sender den til LSTM-laget, og pakker deretter resultatet ut igjen.

> **Merk**: Vi bruker faktisk ikke det utpakkede resultatet `x`, fordi vi bruker output fra de skjulte lagene i de følgende beregningene. Derfor kan vi fjerne utpakkingen helt fra denne koden. Grunnen til at vi plasserer det her er for at du enkelt skal kunne modifisere denne koden, i tilfelle du skulle trenge å bruke nettverksoutput i videre beregninger.


In [7]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

In [8]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.285625
6400: acc=0.33359375
9600: acc=0.3876041666666667
12800: acc=0.44078125
16000: acc=0.4825
19200: acc=0.5235416666666667
22400: acc=0.5559821428571429
25600: acc=0.58609375
28800: acc=0.6116666666666667
32000: acc=0.63340625
35200: acc=0.6525284090909091
38400: acc=0.668515625
41600: acc=0.6822596153846154
44800: acc=0.6948214285714286
48000: acc=0.7052708333333333
51200: acc=0.71521484375
54400: acc=0.7239889705882353
57600: acc=0.7315277777777778
60800: acc=0.7388486842105263
64000: acc=0.74571875
67200: acc=0.7518303571428572
70400: acc=0.7576988636363636
73600: acc=0.7628940217391305
76800: acc=0.7681510416666667
80000: acc=0.7728125
83200: acc=0.7772235576923077
86400: acc=0.7815393518518519
89600: acc=0.7857700892857142
92800: acc=0.7895043103448276
96000: acc=0.7930520833333333
99200: acc=0.7959072580645161
102400: acc=0.798994140625
105600: acc=0.802064393939394
108800: acc=0.8051378676470589
112000: acc=0.8077857142857143
115200: acc=0.8104600694444445
118400

(0.029785829671223958, 0.8138166666666666)

## Toveis og flerlags RNN-er

I våre eksempler har alle rekurrente nettverk operert i én retning, fra begynnelsen av en sekvens til slutten. Det virker naturlig, fordi det ligner på måten vi leser og lytter til tale. Men siden vi i mange praktiske tilfeller har tilfeldig tilgang til inngangssekvensen, kan det være fornuftig å utføre rekurrent beregning i begge retninger. Slike nettverk kalles **toveis** RNN-er, og de kan opprettes ved å sende parameteren `bidirectional=True` til RNN/LSTM/GRU-konstruktøren.

Når vi arbeider med et toveis nettverk, trenger vi to skjulte tilstandsvektorer, én for hver retning. PyTorch koder disse vektorene som én vektor med dobbelt så stor størrelse, noe som er ganske praktisk, fordi du vanligvis sender den resulterende skjulte tilstanden til et fullt tilkoblet lineært lag, og du trenger bare å ta denne økningen i størrelse med i betraktning når du oppretter laget.

Et rekurrent nettverk, enten det er énveis eller toveis, fanger visse mønstre innen en sekvens og kan lagre dem i tilstandsvektoren eller sende dem til utgangen. Som med konvolusjonsnettverk kan vi bygge et annet rekurrent lag oppå det første for å fange mønstre på høyere nivå, bygget fra lavnivåmønstre som det første laget har hentet ut. Dette leder oss til begrepet **flerlags RNN**, som består av to eller flere rekurrente nettverk, der utgangen fra det forrige laget sendes til det neste laget som inngang.

![Bilde som viser et flerlags lang-korttidsminne-RNN](../../../../../translated_images/no/multi-layer-lstm.dd975e29bb2a59fe.webp)

*Bilde fra [denne fantastiske artikkelen](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) av Fernando López*

PyTorch gjør det enkelt å konstruere slike nettverk, fordi du bare trenger å sende parameteren `num_layers` til RNN/LSTM/GRU-konstruktøren for automatisk å bygge flere lag med rekurrens. Dette vil også bety at størrelsen på den skjulte/tilstandsvektoren vil øke proporsjonalt, og du må ta dette med i betraktning når du håndterer utgangen fra de rekurrente lagene.


## RNN-er for andre oppgaver

I denne enheten har vi sett at RNN-er kan brukes til sekvensklassifisering, men faktisk kan de håndtere mange flere oppgaver, som tekstgenerering, maskinoversettelse og mer. Vi vil se nærmere på disse oppgavene i neste enhet.



---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi streber etter nøyaktighet, vær oppmerksom på at automatiserte oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
